# Connecting to camera

Never have multiple notebooks and kernels reading the video stream from a camera at the same time as they might start conflicting with each other

In [ ]:
import cv2

print(F'cv2 version = {cv2.__version__}')

# To find the id of the desired camera, execute
# $ v4l2-ctl --list-devices
# The output will contain the name of the camera and its matching device(s), in form /dev/videoN
# This N is the device ID
deviceId = 0

# cv2a.videoio_registry.getBackends() returns list of all available backends.
availableBackends = [cv2.videoio_registry.getBackendName(b) for b in cv2.videoio_registry.getBackends()]
print(availableBackends)

# Returns list of available backends which works via cv::VideoCapture(int index)
availableCameraBackends = [cv2.videoio_registry.getBackendName(b) for b in cv2.videoio_registry.getCameraBackends()]
print(availableBackends)

# output: ['FFMPEG', 'GSTREAMER', 'CV_IMAGES', 'CV_MJPEG']

# capture
# cap = cv2.VideoCapture(deviceId)
#videoCaptureApi = cv2.CAP_ANY        # autodetect default API
#videoCaptureApi = cv2.CAP_FFMPEG
#videoCaptureApi = cv2.CAP_GSTREAMER 
#videoCaptureApi = cv2.CAP_V4L2 
#videoCaptureApi = cv2.CAP_IMAGES
videoCaptureApi = cv2.CAP_OPENCV_MJPEG
cap = cv2.VideoCapture("/dev/video2", videoCaptureApi)

# cap is of type cv2.VideoCapture
print(f'type(cap) = {type(cap)}')

if not cap.isOpened():
    raise RuntimeError("ERROR! Unable to open camera")

try:
    # width and height of the capture is required for manipulating the image
    # # get() returns float e.g. 1080.0 and we can cast it to int via `int()`
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    heigth = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    while True:
        # A capture is a series of images.
        # Each camera has a property called capture frame rate.
        # It denotes how frequently camera captures the frame. 
        # Its measurement unit is Frames Per Second FPS (e.g. 60 fps).
        # An image buffer is updated each time a new capture occurs.
        # A frame is a single image and we can read it from buffer. 
        # Once we get the frame, we can process it just like we processed
        # images loaded from the disk.
        #
        # tuple unpacking
        ret, frame = cap.read()

        # convert a frame into grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame', gray)

        # press 'q' for exit:
        #    wait 1 millisecond and check if 'q' was pressed
        #    ord() returns the integer that represents the character
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:        
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
import os
import tempfile
import subprocess
import cv2
import numpy as np

# FFMPEG_BIN = '/usr/bin/ffmpeg'
# To get this path execute:
#    $ which ffmpeg
FFMPEG_BIN = '/home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg'

# to find allowed formats 
#    $ ffmpeg -f v4l2 -list_formats all -i /dev/video3
#    ...
#    [video4linux2,v4l2 @ 0x5608ac90af40] Raw: yuyv422: YUYV 4:2:2: 640x480 1280x720 960x544 800x448 640x360 424x240 352x288 320x240 800x600 176x144 160x120 1280x800
#    ...

# /home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg -i /dev/video2 -video_size 640x480 -r 1 -pix_fmt bgr24 -vcodec rawvideo -an -sn -f v4l2
def run_ffmpeg():
    ffmpg_cmd = [
        FFMPEG_BIN,
        '-i', '/dev/video3',
        # '-framerate', '25',
        '-video_size', '640x480',
        # '-r', '1',                # framerate (fps, default is 25)
        '-pix_fmt', 'bgr24',        # opencv requires bgr24 pixel format. 'yuyv422'
        '-vcodec', 'rawvideo',
        '-an','-sn',                # disable audio processing
        # '-f', 'v4l2',
        '-f', 'image2pipe',
        '-',                        # output to go to stdout
    ]
    return subprocess.Popen(ffmpg_cmd, stdout = subprocess.PIPE, bufsize=10**8)

def run_cv_window(process):
    while True:
        # Capture frame-by-frame
        raw_image = process.stdout.read(640*480*3)
        # print(type(raw_image))
        # print(raw_image) # only for debugging but it might make browser unresponsive due to large amount of data being appended to the display in this tab
        if raw_image == b'':
            raise RuntimeError("Empty pipe")
        
        # transform the byte read into a numpy array
        frame =  np.frombuffer(raw_image, dtype='uint8')
        frame = frame.reshape((480,640,3))          # height, width, colours
        if frame is not None:
            # cv2.imshow('Video', frame) # uncomment to show the original image
            
            # example how to edit captured frame and then display it
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('frame', gray)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        process.stdout.flush()
    
    cv2.destroyAllWindows()
    process.terminate()
    print(process.poll())


def run():
    ffmpeg_process = run_ffmpeg()
    run_cv_window(ffmpeg_process)

run()